In [1]:
import torch
from src.model.spex_plus import SpExPlus
from src.datasets.mix_dataset import MixDataset
from src.metric.utils import si_sdr
import pesq
model_path = "saved/models/default_config/1108_065537/checkpoint-epoch80.pth"
dataset = MixDataset("/home/vladimir/PycharmProjects/TTS/mixtures_data/test_clean",None, None)
model = SpExPlus(num_speakers=96).cuda()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda'))['state_dict'])

<All keys matched successfully>

In [24]:
mix_audio = obj['mix_audio']
frame_size = 3 * 16000 
padded_audio = torch.nn.functional.pad(mix_audio, (0, frame_size - mix_audio.size(1) % frame_size))
mix_audio = padded_audio.reshape(-1, frame_size)

In [25]:
target_audio = obj['target_audio']
padded_audio = torch.nn.functional.pad(target_audio, (0, frame_size - target_audio.size(1) % frame_size))
target_audio = padded_audio.reshape(-1, frame_size)

In [26]:
reference_audio = obj['reference_audio'].repeat(len(mix_audio), 1)
reference_audio_len = torch.tensor([reference_audio.shape[1]]).repeat(len(mix_audio), 1).flatten()

In [27]:
pred = model(mix_audio.cuda(), reference_audio.cuda(), reference_audio_len.cuda())['s1'].detach().cpu().numpy()
target = target_audio.detach().numpy()

In [28]:
pesq.pesq(16_000, pred.flatten(), target.flatten())

1.283005952835083

In [29]:
audio_len = obj['target_audio_len'].item()

In [30]:
import numpy as np

def si_sdr(est, target):
    alpha = np.sum(target * est) / np.linalg.norm(target) ** 2
    return 20 * np.log10(
        np.linalg.norm(alpha * target)
        / (np.linalg.norm(alpha * target - est) + 1e-6)
        + 1e-6
    )

In [31]:
si_sdr(pred.flatten(), target.flatten())

5.077021033334357

In [145]:
from scipy.signal import wiener


In [148]:
# Преобразование в частотное представление
fft_wavka = torch.fft.fft(torch.tensor(pred.flatten()))

# Обратное преобразование во временное представление
ifft_wavka = torch.fft.ifft(fft_wavka).real

In [155]:
import torchaudio.transforms as T

spectral_subtract = T.HighPassFilter(16_000)
denoised_waveform = spectral_subtract(ifft_wavka)

AttributeError: module 'torchaudio.transforms' has no attribute 'HighPassFilter'

In [150]:
si_sdr(wiener(ifft_wavka.numpy()), target.flatten())

5.070686374886655

In [141]:
from IPython.display import Audio

In [151]:
Audio(data=wiener(pred.flatten()), rate=16_000)

In [152]:
pesq.pesq(16_000, wiener(pred.flatten()), target.flatten())

1.2881510257720947